# Combination Generator

In [298]:
import pandas as pd
import numpy as np
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.1")
db = lusas.database()

Read the excel definitions

In [299]:
file_name = "12 Combinations.xlsx"
actions_sheet = "Actions"
do_print = False # Print out each of the input tables for debugging
is610 = True    # Create combination 6.10 or 6.10a and 6.10b

In [300]:
df_permanent = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(0,4), header=1).dropna()
if do_print : print(df_permanent.head())

In [301]:
df_variable = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(5,10), header=1).dropna()
if do_print : print(df_variable.head())

In [302]:
df_prestress = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(11,14), header=1).dropna()
df_prestress.columns = df_prestress.columns.str.replace(r'.\d+', '', regex=True)
if do_print : print(df_prestress.head())

In [303]:
df_accidental = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(15,17), header=1).dropna()
if do_print : print(df_accidental.head())

In [304]:
df_perm_loadcases = pd.read_excel(file_name, sheet_name="PermanentLoadcases")
if do_print : print(df_perm_loadcases.head())
dict_perm_loadcases = df_perm_loadcases.set_index('Action Name').T.to_dict('list')
if do_print : print(dict_perm_loadcases)

In [305]:
df_var_loadcases = pd.read_excel(file_name, sheet_name="VariableLoadcases")
if do_print : print(df_var_loadcases.head())
dict_var_loadcases = df_var_loadcases.set_index('Action Name').T.to_dict('list')

In [306]:
df_acc_loadcases = pd.read_excel(file_name, sheet_name="AccidentalLoadcases")
if do_print : print(df_acc_loadcases.head())

In [307]:
# Helper method to add different loadcase types to a smart combination
def add_loadset_to_smart_comb(smart_comb, loadcase:str, beneficial:float, adverse:float) -> bool:
    if db.existsLoadset(loadcase):
        loadset = db.getLoadset(loadcase)
        type_code = loadset.getTypeCode()

        if type_code <= 2:
            smart_comb.addEntry(beneficial, (adverse-beneficial), loadset)
        elif type_code == 3:
            envelope1 = win32.CastTo(loadset, "IFEnvelope")
            envelope2 = envelope1.getAssocLoadset()
            smart_comb.addEntry(beneficial, (adverse-beneficial), envelope1)
            smart_comb.addEntry(beneficial, (adverse-beneficial), envelope2)
        elif type_code == 6:
            smart1 = win32.CastTo(loadset, "IFSmartCombination")
            smart2 = smart1.getAssocLoadset()
            smart_comb.addEntry(beneficial, (adverse-beneficial), smart1)
            smart_comb.addEntry(beneficial, (adverse-beneficial), smart2)
        return True
    else:
        return False


In [308]:
# Helper function to add permanent actions to the smart combination
def add_permanent_actions_to_smart_combination(smart_comb, include_reduction=False):
    for action_type in dict_perm_loadcases.keys():
        # Factors for this type
        beneficial = df_permanent[df_permanent['Perm Action Name'] == action_type]['Beneficial']
        adverse    = df_permanent[df_permanent['Perm Action Name'] == action_type]['Adverse']
        reduction  = df_permanent[df_permanent['Perm Action Name'] == action_type]['Reduction']
        reduction  = reduction if include_reduction else 1.0

        # LUSAS Loadcases
        for loadcase in dict_perm_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if not add_loadset_to_smart_comb(smart_comb, loadcase, beneficial, adverse * reduction):
                print(f"Permanent loadcase '{loadcase}' does not exists")

In [309]:
# Helper function to add factored prestress loads to the smart combination
def add_prestress_loads_to_smart_combination(smart_comb):
    for ip, rowp in df_prestress.iterrows():
        loadcase = rowp['Loadcase Name']
        if not isinstance(loadcase, str) and np.isnan(loadcase) : continue        
        
        # Factors for this type
        beneficial = rowp['Beneficial']
        adverse    = rowp['Adverse']

        if not add_loadset_to_smart_comb(smart_comb, loadcase, beneficial, adverse):
            print(f"Prestress loadcase '{loadcase}' does not exists")

## Equ 6.10

$$ \sum_{j\ge1}{\gamma_{G,j}G_{k,j} + \gamma_pP + \gamma_{Q,1}Q_{k,1}}  + \sum_{i\gt1}{ \gamma_{Q,i}\psi_{0,i}Q_{k,i}}

In [310]:
if is610:

    uls_envelope = db.createEnvelope("ULS 6.10")
    uls_envelope.setTreeLocation("Design Envelopes", True)

    # Consider each variable action as a leading action
    for i, row in df_variable.iterrows():
        action = row["Var Action Name"]
        load_factor = row["DesignFactor"]
        comb_factor = row["CombinationFactor"]
        
        smart_comb = db.createCombinationSmart(f"ULS - 610 - {action}")

        # Permanent actions
        add_permanent_actions_to_smart_combination(smart_comb)

        # Prestress actions
        add_prestress_loads_to_smart_combination(smart_comb)

        # Variable actions
        for action_type in dict_var_loadcases.keys():
            # Determine variable factor
            if action_type == action:
                adverse = load_factor                        # Leading Variable
            else:
                adverse = load_factor * comb_factor          # Accompanying Variable

            # LUSAS Loadcases
            for loadcase in dict_var_loadcases[action_type]:
                if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

                if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, adverse):
                    print(f"Variable loadcase '{loadcase}' does not exists")


        # Add combination to overall design envelope
        uls_envelope.addEntry(smart_comb)
        uls_envelope.addEntry(smart_comb.getAssocLoadset())
        smart_comb.setTreeParent(uls_envelope)
        smart_comb.getAssocLoadset().setTreeParent(uls_envelope.getAssocLoadset())


## 6.10a Or 6.10b

$$ \sum_{j\ge1}{\gamma_{G,j}G_{k,j} + \gamma_pP + \gamma_{Q,1}\psi_{0,i}Q_{k,1}}  + \sum_{i\gt1}{ \gamma_{Q,i}\psi_{0,i}Q_{k,i}}

In [311]:
if not is610:
    uls_envelope = db.createEnvelope("ULS 6.10a or 6.10b")
    uls_envelope.setTreeLocation("Design Envelopes", True)

    uls_envelope_610a = db.createEnvelope("ULS 6.10a")
    uls_envelope_610b = db.createEnvelope("ULS 6.10b")

    uls_envelope.addEntry(uls_envelope_610a)
    uls_envelope.addEntry(uls_envelope_610a.getAssocLoadset())

    uls_envelope.addEntry(uls_envelope_610b)
    uls_envelope.addEntry(uls_envelope_610b.getAssocLoadset())

    uls_envelope_610a.setTreeParent(uls_envelope)
    uls_envelope_610a.getAssocLoadset().setTreeParent(uls_envelope.getAssocLoadset())
        
    uls_envelope_610b.setTreeParent(uls_envelope)
    uls_envelope_610b.getAssocLoadset().setTreeParent(uls_envelope.getAssocLoadset())


In [312]:
if not is610:
    # Expression 610a
    # All variable actions consider a combination factor
    smart_comb = db.createCombinationSmart(f"ULS - 610a")

    # Permanent actions
    add_permanent_actions_to_smart_combination(smart_comb)

    # Prestress actions
    add_prestress_loads_to_smart_combination(smart_comb)

    # Variable actions
    for i, row in df_variable.iterrows():
        action_type = row["Var Action Name"]
        load_factor = row["DesignFactor"]
        comb_factor = row["CombinationFactor"]

        # Add all of the LUSAS Loadcases of the action type
        for loadcase in dict_var_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, load_factor * comb_factor):
                print(f"Variable loadcase '{loadcase}' does not exists")


    # Add combination to 6.10a envelope
    uls_envelope_610a.addEntry(smart_comb)
    uls_envelope_610a.addEntry(smart_comb.getAssocLoadset())
    smart_comb.setTreeParent(uls_envelope_610a)
    smart_comb.getAssocLoadset().setTreeParent(uls_envelope_610a.getAssocLoadset())

C:\Users\rob\AppData\Local\Temp\gen_py\3.12\AC0CF861-2101-11CF-91D8-0020AFC19F64x0x21x1\IFSmartCombination.py:46: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ret = self._oleobj_.InvokeTypes(1610874881, LCID, 1, (9, 0), ((5, 1), (5, 1), (12, 1), (12, 17), (12, 17), (12, 17)),factor


Exp(610b)

$$ \sum_{j\ge1}{\xi_j\gamma_{G,j}G_{k,j} + \gamma_pP + \gamma_{Q,1}Q_{k,1}}  + \sum_{i\gt1}{ \gamma_{Q,i}\psi_{0,i}Q_{k,i}}

In [313]:
if not is610:
    # Expression 610b
    # Consider each variable action as a leading action
    for i, row in df_variable.iterrows():
        action = row["Var Action Name"]
        load_factor = row["DesignFactor"]
        comb_factor = row["CombinationFactor"]
        
        smart_comb = db.createCombinationSmart(f"ULS - 6.10b - {action}")

        # Permanent actions with associated reduction factor
        add_permanent_actions_to_smart_combination(smart_comb, include_reduction=True)

        # Prestress actions
        add_prestress_loads_to_smart_combination(smart_comb)

        # Variable actions
        for action_type in dict_var_loadcases.keys():
            # Determine variable factor
            if action_type == action:
                adverse = load_factor                        # Leading Variable
            else:
                adverse = load_factor * comb_factor          # Accompanying Variable

            # Add all of the LUSAS Loadcases of the action type
            for loadcase in dict_var_loadcases[action_type]:
                if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

                if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, adverse):
                    print(f"Variable loadcase '{loadcase}' does not exists")


        # Add leading variable action combination to 6.10b envelope
        uls_envelope_610b.addEntry(smart_comb)
        uls_envelope_610b.addEntry(smart_comb.getAssocLoadset())
        smart_comb.setTreeParent(uls_envelope_610b.getAssocLoadset())
        smart_comb.getAssocLoadset().setTreeParent(uls_envelope_610b)